# LangChain による Bedrock - あるプログラミング言語から別のプログラミング言語へのコード変換

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## はじめに

このノートブックでは、Amazon Bedrock 上で LLM を使ってあるプログラミング言語から別のプログラミング言語へコードを変換する方法を学びます。LLM の使い方と、LangChain フレームワークを利用して Bedrock と統合する方法を説明します。

このラボでは、Amazon Bedrock の Claude v2 モデルを使用します。

**注意:** *このノートブックは AWS 環境内外で実行できます*

#### コンテキスト
前の例 `02_code_interpret_w_langchain.ja.ipynb` では、LangChain フレームワークを使って Amazon Bedrock API と通信する方法を探りました。前回のコード解釈/説明の例と同様に、LangChain と Amazon Bedrock API を使って、あるレガシープログラミング言語から別のプログラミング言語へコードを翻訳します。


#### パターン
Amazon Bedrock API の LangChain 実装に、タスク、命令、入力からなる入力を提供するだけで、追加の例を提供することなく、モデルが出力を生成します。ここでの目的は、強力な LLM がいかに簡単に手元のタスクを理解し、説得力のある出力を生成するかを示すことです。

![](./images/code-translation-langchain.png)

#### ユースケース
Amazon Bedrock LLM を使って、あるプログラミング言語から別のプログラミング言語へコードを翻訳する方法を紹介します。

#### ペルソナ
Amazon Bedrock と LangChain API を使用して、C++ コードを Java に変換する方法を説明します。言語間の構文、言語構成、規約の違いを扱いながら、C++ コードを Java に移植するモデルを促すテクニックを示します。

#### 実装
このユースケースを実現するために、与えられたレガシー C++ コードを Java に移植する翻訳方法を紹介します。 Amazon Bedrock と LangChain の統合を使います。

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して（必要なライブラリーがインストールされていることを確認し）Bedrock に接続する必要があります。

セットアップの仕組みや ⚠️ **変更が必要かどうかの詳細**については [Bedrock boto3 setup ノートブック](../00_Intro/bedrock_boto3_setup.ipynb)を参照してください。

このノートブックでは [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) ライブラリもインストールします。このライブラリは、入力プロンプトのトークン数をカウントするのに使います。

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.309 "transformers>=4.24,<5"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## Bedrock を実行する

まずは LLM から Bedrock クラスのインスタンスを作成します。ここでは Amazon Bedrock で利用可能なモデルの ARN である `model_id` を指定します。

オプションで、`temperature`, `topP`, `maxTokenCount` や `stopSequences` などのパラメータを始めとする `model_kwargs` を、以前に作成した boto3 クライアントを同様、渡すことができます（パラメータの詳細は Amazon Bedrock コンソールで確認可能です）。

Amazon Bedrock で利用可能なテキスト生成モデルの ID は以下の通りです:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v2


モデルによってサポートする `model_kwargs` が異なることに注意してください。

In [4]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## LangChain カスタムプロンプトテンプレートの作成

プロンプトのテンプレートを作成することで、実行のたびに異なる入力変数をプロンプトに渡すことができます。これは、データベースから取得した異なる入力変数を使ってコンテンツを生成しなければならない場合に便利です。

In [5]:
# C++ で書かれた車両管理コード
sample_code = """
#include <iostream>
#include <string>
#include <vector>

class Vehicle {
protected:
    std::string registrationNumber;
    int milesTraveled;
    int lastMaintenanceMile;

public:
    Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}

    virtual void addMiles(int miles) {
        milesTraveled += miles;
    }

    virtual void performMaintenance() {
        lastMaintenanceMile = milesTraveled;
        std::cout << "Maintenance performed for vehicle: " << registrationNumber << std::endl;
    }

    virtual void checkMaintenanceDue() {
        if ((milesTraveled - lastMaintenanceMile) > 10000) {
            std::cout << "Vehicle: " << registrationNumber << " needs maintenance!" << std::endl;
        } else {
            std::cout << "No maintenance required for vehicle: " << registrationNumber << std::endl;
        }
    }

    virtual void displayDetails() = 0;

    ~Vehicle() {
        std::cout << "Destructor for Vehicle" << std::endl;
    }
};

class Truck : public Vehicle {
    int capacityInTons;

public:
    Truck(std::string regNum, int capacity) : Vehicle(regNum), capacityInTons(capacity) {}

    void displayDetails() override {
        std::cout << "Truck with Registration Number: " << registrationNumber << ", Capacity: " << capacityInTons << " tons." << std::endl;
    }
};

class Car : public Vehicle {
    std::string model;

public:
    Car(std::string regNum, std::string carModel) : Vehicle(regNum), model(carModel) {}

    void displayDetails() override {
        std::cout << "Car with Registration Number: " << registrationNumber << ", Model: " << model << "." << std::endl;
    }
};

int main() {
    std::vector<Vehicle*> fleet;

    fleet.push_back(new Truck("XYZ1234", 20));
    fleet.push_back(new Car("ABC9876", "Sedan"));

    for (auto vehicle : fleet) {
        vehicle->displayDetails();
        vehicle->addMiles(10500);
        vehicle->checkMaintenanceDue();
        vehicle->performMaintenance();
        vehicle->checkMaintenanceDue();
    }

    for (auto vehicle : fleet) {
        delete vehicle; 
    }

    return 0;
}
"""

In [6]:
from langchain import PromptTemplate

# 複数の入力変数を持つプロンプト・テンプレートを作成する
multi_var_prompt = PromptTemplate(
    input_variables=["code", "srcProgrammingLanguage", "targetProgrammingLanguage"], 
    template="""

Human: You will be acting as an expert software developer in {srcProgrammingLanguage} and {targetProgrammingLanguage}. 
You will tranlslate below code from {srcProgrammingLanguage} to {targetProgrammingLanguage} while following coding best practices.
<code>
{code}
</code>

Assistant: """
)

# 入力変数に値を渡す
prompt = multi_var_prompt.format(code=sample_code, srcProgrammingLanguage="C++", targetProgrammingLanguage="Java")


### C++ からJavaへのコード変換

In [ ]:
response = textgen_llm(prompt)

target_code = response[response.index('\n')+1:]

print_ww(target_code)

## まとめ

この例では、Amazon Bedrock と LangChain を使って、単純なテキストプロンプトでレガシー C++ プログラムを Java に変換する方法を学びました。